## 导入数据,并转换

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

df_data_1_C_plot = pd.read_csv("data/df_data_1_C_plot.csv")
df_data_1_C_plot.columns

df_data_1_C_plot = df_data_1_C_plot[df_data_1_C_plot['转炉终点Mn'] != 0]
print("when use 转炉终点Mn != 0 :", len(df_data_1_C_plot))
# df_data_1_C_plot.columns


df_data_1_C_plot_future = df_data_1_C_plot[['转炉终点温度', '转炉终点C', '转炉终点Mn', '转炉终点S', '转炉终点P', '转炉终点Si',
                                            '钢水净重', '氮化钒铁FeV55N11-A', '低铝硅铁', '钒氮合金(进口)',
                                            '钒铁(FeV50-B)', '硅铝钙', '硅铝合金FeAl30Si25',
                                            '硅锰面（硅锰渣）', '硅铁(合格块)', '硅铁FeSi75-B', '石油焦增碳剂',
                                            '锰硅合金FeMn64Si27(合格块)', '锰硅合金FeMn68Si18(合格块)', '碳化硅(55%)', '硅钙碳脱氧剂']]

df_data_1_C_plot_label = df_data_1_C_plot['C_Yield']

when use 转炉终点Mn != 0 : 218


In [3]:
df_data_1_Mn_plot = pd.read_csv("data/df_data_1_Mn_plot.csv")
# df_data_1_Mn_plot.columns

df_data_1_Mn_plot_future = df_data_1_Mn_plot[['转炉终点温度', '转炉终点C', '转炉终点Mn', '转炉终点S', '转炉终点P', '转炉终点Si',
                                              '钢水净重', '氮化钒铁FeV55N11-A', '低铝硅铁', '钒氮合金(进口)',
                                              '钒铁(FeV50-B)', '硅铝钙', '硅铝合金FeAl30Si25',
                                              '硅锰面（硅锰渣）', '硅铁(合格块)', '硅铁FeSi75-B', '石油焦增碳剂',
                                              '锰硅合金FeMn64Si27(合格块)', '锰硅合金FeMn68Si18(合格块)', '碳化硅(55%)', '硅钙碳脱氧剂']]

df_data_1_Mn_plot_label = df_data_1_Mn_plot['Mn_Yield']

## 样本中不同钢号的数量统计

In [4]:
for i in set(df_data_1_C_plot['钢号']):
    print(i," : ",len(df_data_1_C_plot[df_data_1_C_plot['钢号'] == i]))

HRB500D      :  18
HRB400D      :  32
HRB400B      :  161
Q345B        :  7


In [5]:
for i in set(df_data_1_Mn_plot['钢号']):
    print(i," : ",len(df_data_1_Mn_plot[df_data_1_Mn_plot['钢号'] == i]))

HRB500D      :  18
HRB400D      :  34
HRB400B      :  164
Q345B        :  6


In [6]:
set(df_data_1_Mn_plot['钢号'])

{'HRB400B    ', 'HRB400D    ', 'HRB500D    ', 'Q345B      '}

### 不同钢号的化学成分国家标准

In [68]:
HRB = {"C":0.25,"Si":0.80,"Mn":1.60,"P":0.045,"S":0.045}
Q35B = {"C":0.20,"Si":0.50,"Mn":1.70,"P":0.035,"S":0.035}

HRB["C"]

0.25

## 加载已经训练好的模型
### C

In [8]:
from sklearn.metrics import mean_squared_error
import xgboost as xgb

C_CV_Model_new = xgb.Booster({'nthread': -1})  # init model
C_CV_Model_new.load_model('model/C_CV_Model.model')  # load data

dpredict = xgb.DMatrix(df_data_1_C_plot_future)
predict = C_CV_Model_new.predict(dpredict)

from sklearn.metrics import mean_squared_error
print("MSE of predict ：", (mean_squared_error(y_true=df_data_1_C_plot_label,y_pred=predict)))

MSE of predict ： 0.0014731971837747403


### 加载Mn的模型

In [9]:
Mn_CV_Model_new = xgb.Booster({'nthread': -1})  # init model
Mn_CV_Model_new.load_model('model/Mn_CV_Model.model')  # load data

dpredict = xgb.DMatrix(df_data_1_Mn_plot_future)
predict = Mn_CV_Model_new.predict(dpredict)

from sklearn.metrics import mean_squared_error
print("MSE of predict ：", (mean_squared_error(y_true=df_data_1_Mn_plot_label,y_pred=predict)))

MSE of predict ： 0.00019115669935288612


## 通过所得率计算最终成分

In [65]:
def Get_Yield(df_sample,element = "C"):
    import xgboost as xgb

    element_CV_Model_new = xgb.Booster({'nthread': -1})  # init model
    element_CV_Model_new.load_model('model/'+ element +'_CV_Model.model')  # load data
    dmatrix_one = xgb.DMatrix(df_sample)
    return element_CV_Model_new.predict(dmatrix_one)

def Get_Content(df_sample,element="C",set_future = set_future):

    Yield = Get_Yield(df_sample=df_sample,element='C')
    
    All_element = 0
    for i in set(set_future):
        All_element += df_sample[i].iloc[0].values[0] * \
            df_money_0[df_money_0['合金配料'] == i][element].values[0]

    final_element = (All_element * Yield +
                     df_sample['钢水净重'].iloc[0].values[0] * df_sample['转炉终点' + element].iloc[0].values[0])
    / df_sample['钢水净重'].iloc[0]
    
#     print("钢水净重 :",df_sample['钢水净重'].iloc[0].values[0])
#     print("投入元素总重 :",All_element)
#     print("收得率 :",Yield[0])
#     print("转炉终点 %s :"%element,df_sample['转炉终点' + element].iloc[0].values[0])
#     print("连铸正样 %s :"%element,final_element[0])
    
    return final_element[0]

In [67]:
set_future = {'氮化钒铁FeV55N11-A', '低铝硅铁', '钒氮合金(进口)',
          '钒铁(FeV50-B)', '硅铝钙', '硅铝合金FeAl30Si25',
          '硅锰面（硅锰渣）', '硅铁(合格块)', '硅铁FeSi75-B', '石油焦增碳剂',
          '锰硅合金FeMn64Si27(合格块)', '锰硅合金FeMn68Si18(合格块)', '碳化硅(55%)', '硅钙碳脱氧剂'}

read_path = "data/money_0.csv"
df_money_0 = pd.read_csv(read_path)
df_money_0 = df_money_0.fillna(0)

list_one = np.ones(len(columns))

def list2df(list_sample):
    columns = ['转炉终点温度', '转炉终点C', '转炉终点Mn', '转炉终点S', '转炉终点P', '转炉终点Si',
                                              '钢水净重', '氮化钒铁FeV55N11-A', '低铝硅铁', '钒氮合金(进口)',
                                              '钒铁(FeV50-B)', '硅铝钙', '硅铝合金FeAl30Si25',
                                              '硅锰面（硅锰渣）', '硅铁(合格块)', '硅铁FeSi75-B', '石油焦增碳剂',
                                              '锰硅合金FeMn64Si27(合格块)', '锰硅合金FeMn68Si18(合格块)', '碳化硅(55%)', '硅钙碳脱氧剂']
    
    df_sample = pd.DataFrame([list_sample])
    df_sample.columns = [columns]
    
    return df_sample

df_sample = list2df(list_one)

print(Get_Content(df_sample,set_future=set_future))

2.1683698


## 通过投入金属计算成本

In [49]:
df_sample = pd.DataFrame([list_one])
df_sample.columns = [columns]
df_money_0.columns

Index(['合金配料', 'C', 'Mn', 'S', 'P', 'Si', 'Al', 'Ca', 'V', 'N', '价格(元/吨)'], dtype='object')

In [55]:
def Get_Cost(df_sample = list_one,set_future = set_future):

    All_Cost = 0
    for i in set(set_future):
        All_Cost += df_sample[i].iloc[0].values[0] * \
            df_money_0[df_money_0['合金配料'] == i]["价格(元/吨)"].values[0] / 1000

    return All_Cost/(df_sample['钢水净重']/1000).iloc[0].values[0]
print(Get_Cost(df_sample,set_future=set_future))

974900.0


## 计算历史数据中每吨钢水所需投入合金元素之平均成本

In [56]:
def Get_Avg_Cost(df_data_1_C_plot = df_data_1_C_plot,set_future = set_future):
    All_Cost = 0
    for i in set(set_future):
        All_Cost += df_data_1_C_plot[i] * \
            df_money_0[df_money_0['合金配料'] == i]["价格(元/吨)"].values[0] / 1000

    return np.mean(All_Cost/(df_data_1_C_plot['钢水净重']/1000))

print(Get_Avg_Cost(df_data_1_C_plot,set_future=set_future)) 

343.06933462317204


## 设定评价函数

individual 为自动生成的族群

```
creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # 定义最大化适应度
creator.create("Individual", list, fitness=creator.FitnessMax)  # 这里的list种群的数据类型

toolbox = base.Toolbox()

# Attribute generator: define 'attr_bool' to be an attribute ('gene')
#                      which corresponds to integers sampled uniformly
#                      from the range [0,1] (i.e. 0 or 1 with equal
#                      probability)


toolbox.register("attr_bool", random.random)  # 包含了0,1的随机整数,初始化种群

# Structure initializers: define 'individual' to be an individual
#                         consisting of 100 'attr_bool' elements ('genes')
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_bool, 5)

# define the population to be a list of 'individual's
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


# create an initial population of 300 individuals (where
# each individual is a list of integers)
pop = toolbox.population(n=300)  # 定义300个个体的种群
```

In [81]:
# list_one -> individual
# Get_Content(individual,element = "C",set_future=set_future)
Avg_Cost = 343.06933462317204
# Get_Cost(individual,,set_future=set_future)


def evalOneMin(individual):

    set_future = {'氮化钒铁FeV55N11-A', '低铝硅铁', '钒氮合金(进口)',
                  '钒铁(FeV50-B)', '硅铝钙', '硅铝合金FeAl30Si25',
                  '硅锰面（硅锰渣）', '硅铁(合格块)', '硅铁FeSi75-B', '石油焦增碳剂',
                  '锰硅合金FeMn64Si27(合格块)', '锰硅合金FeMn68Si18(合格块)', '碳化硅(55%)', '硅钙碳脱氧剂'}

    df_individual = list2df(individual)

    HRB = {"C": 0.25, "Si": 0.80, "Mn": 1.60, "P": 0.045, "S": 0.045}

    C_sub_percentage = np.abs(
        (Get_Content(df_individual, element="C", set_future=set_future) - HRB["C"])/HRB["C"])
    Mn_sub_percentage = np.abs((Get_Content(
        df_individual, element="Mn", set_future=set_future) - HRB["Mn"])/HRB["C"])
    Si_sub_percentage = np.abs((Get_Content(
        df_individual, element="Si", set_future=set_future) - HRB["Si"])/HRB["C"])

    All_Cost_sub_percentage = (
        Get_Cost(df_individual, set_future=set_future) - Avg_Cost)/Avg_Cost

    sums = All_Cost_sub_percentage + C_sub_percentage + \
        Mn_sub_percentage + Si_sub_percentage

    return sums,

In [82]:
evalOneMin(list_one)

(2865.748599783453,)

## 定义目标函数

问题三: 实现钢水脱氧合金化成本优化计算,并给出合金配料方案


此问,我们通过遗传算法来实现,我们需要根据需求来设定目标函数.首先由于钢材本身的化学成分标准与题目中钢厂产钢目标值.我们需要将脱氧合金化后的钢水重量与目标结果尽量小.在此同时尽量降低成本.

因此首先我们需要不同元素收得率的预测模型,经过计算我们发现其中P,S无法进行稳定预测.因此我们将目标函数的中元素化学成分部分,元素种类限制为,C,Mn,Si三种元素.

同时我们还需要一个总成本计算函数.

不完美之处:
    身为非专业者,目标函数中三种元素含量与成本之间的权重不好设立.